# Implementing CNN for cat or non-Cat

## Import some Libraries

In [7]:
#Import some libraries
from keras.models import Sequential
import glob
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from skimage import transform
import os

### Colleting Dataset

In [8]:
width = 68
height = 68
X = []
y = []
def get_dataset(path_image, Cat=True):
    images = glob.glob(path_image+"*.png")
    
    for image in images:
        img = Image.open(image)
        img = np.asarray(img)
        if Cat:
            y.append(1)
        else:
            y.append(0)
        img = transform.resize(img,(width, height))
        X.append(img)
        
path_images_cat = 'train/cat/'
path_images_n_cat = 'train/noncat/'

get_dataset(path_images_cat)
get_dataset(path_images_n_cat, False)

X = np.asarray(X)
y = np.asarray(y)

In [9]:
# check the shape of data set and labels
print("Shape of X: {}".format(X.shape))
print("Shape of y: {}".format(y.shape))

Shape of X: (209, 68, 68, 3)
Shape of y: (209,)


## Build the CNN model

In [35]:
epochs = 500
batch_size = 128
BATCH_NORM = False
num_class = 2

In [14]:
# Sequential model

In [27]:
#Stacking layes
from keras.models import Sequential
from keras.layers.convolutional import Convolution2D, MaxPooling2D, Conv2D
from keras.layers import Flatten, Dense, Activation, Dropout
from keras.optimizers import Adam, RMSprop, SGD
from keras.layers.normalization import BatchNormalization


model = Sequential()

model.add(Conv2D(128, (3, 3), padding='same', input_shape=X.shape[1:]))
model.add(BatchNormalization()) if BATCH_NORM else None
model.add(Activation('relu'))

model.add(Conv2D(32, (3, 3), padding='same'))
model.add(BatchNormalization()) if BATCH_NORM else None
model.add(Activation('relu'))

model.add(Flatten())

model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(num_class))
model.add(Activation('softmax'))

## Set the optimizer & compile the model

In [28]:
# set learning rate
lr = 0.001
opt = RMSprop(lr)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [29]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 68, 68, 128)       3584      
_________________________________________________________________
activation_7 (Activation)    (None, 68, 68, 128)       0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 68, 68, 32)        36896     
_________________________________________________________________
activation_8 (Activation)    (None, 68, 68, 32)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 147968)            0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               37880064  
_________________________________________________________________
activation_9 (Activation)    (None, 256)              

### Train Test Split

In [32]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=5)

X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=5)


### Transfor Vector to Matrix

In [34]:
from keras.utils import np_utils
print("Shape of y_train: {}\t Shape of y_val: {}\tShape of y_test: {}".format(y_train.shape, y_val.shape, y_test.shape))

y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
y_val = np_utils.to_categorical(y_val)

print("Shape of y_train: {}\t Shape of y_val: {}\tShape of y_test: {}".format(y_train.shape, y_val.shape, y_test.shape))

Shape of y_train: (133,)	 Shape of y_val: (34,)	Shape of y_test: (42,)
Shape of y_train: (133, 2)	 Shape of y_val: (34, 2)	Shape of y_test: (42, 2)


In [37]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, 
          validation_data=(X_val, y_val), shuffle=True)


Train on 133 samples, validate on 34 samples
Epoch 1/500
133/133 [==============================] - 5s 35ms/step - loss: 8.8025 - accuracy: 0.6241 - val_loss: 7.7855 - val_accuracy: 0.3235
Epoch 2/500
133/133 [==============================] - 3s 25ms/step - loss: 7.2400 - accuracy: 0.3759 - val_loss: 3.3855 - val_accuracy: 0.6765
Epoch 3/500
133/133 [==============================] - 3s 25ms/step - loss: 3.4480 - accuracy: 0.6241 - val_loss: 0.6965 - val_accuracy: 0.6765
Epoch 4/500
133/133 [==============================] - 3s 25ms/step - loss: 0.6637 - accuracy: 0.6316 - val_loss: 0.6547 - val_accuracy: 0.6765
Epoch 5/500
133/133 [==============================] - 3s 26ms/step - loss: 0.6433 - accuracy: 0.6541 - val_loss: 0.6424 - val_accuracy: 0.6765
Epoch 6/500
133/133 [==============================] - 4s 29ms/step - loss: 0.6349 - accuracy: 0.6316 - val_loss: 1.0825 - val_accuracy: 0.6765
Epoch 7/500
133/133 [==============================] - 4s 26ms/step - loss: 1.4591 - accur

Epoch 56/500
133/133 [==============================] - 4s 31ms/step - loss: 0.2753 - accuracy: 0.8647 - val_loss: 0.5074 - val_accuracy: 0.6765
Epoch 57/500
133/133 [==============================] - 4s 31ms/step - loss: 0.2280 - accuracy: 0.8722 - val_loss: 0.4822 - val_accuracy: 0.6765
Epoch 58/500
133/133 [==============================] - 4s 31ms/step - loss: 0.2028 - accuracy: 0.8872 - val_loss: 2.0177 - val_accuracy: 0.6765
Epoch 59/500
133/133 [==============================] - 4s 31ms/step - loss: 1.4885 - accuracy: 0.6767 - val_loss: 4.1250 - val_accuracy: 0.6471
Epoch 60/500
133/133 [==============================] - 4s 31ms/step - loss: 2.8262 - accuracy: 0.6767 - val_loss: 0.6102 - val_accuracy: 0.7059
Epoch 61/500
133/133 [==============================] - 4s 31ms/step - loss: 0.4385 - accuracy: 0.7895 - val_loss: 0.4704 - val_accuracy: 0.7647
Epoch 62/500
133/133 [==============================] - 5s 35ms/step - loss: 0.1832 - accuracy: 0.8947 - val_loss: 1.7824 - val_ac

133/133 [==============================] - 4s 33ms/step - loss: 0.0104 - accuracy: 1.0000 - val_loss: 0.3246 - val_accuracy: 0.8529
Epoch 113/500
133/133 [==============================] - 4s 33ms/step - loss: 0.0077 - accuracy: 1.0000 - val_loss: 0.3426 - val_accuracy: 0.8824
Epoch 114/500
133/133 [==============================] - 4s 31ms/step - loss: 0.0075 - accuracy: 1.0000 - val_loss: 0.3396 - val_accuracy: 0.8824
Epoch 115/500
133/133 [==============================] - 4s 31ms/step - loss: 0.0070 - accuracy: 1.0000 - val_loss: 0.3499 - val_accuracy: 0.8824
Epoch 116/500
133/133 [==============================] - 5s 36ms/step - loss: 0.0045 - accuracy: 1.0000 - val_loss: 0.3692 - val_accuracy: 0.8529
Epoch 117/500
133/133 [==============================] - 4s 31ms/step - loss: 0.0068 - accuracy: 1.0000 - val_loss: 0.3508 - val_accuracy: 0.8529
Epoch 118/500
133/133 [==============================] - 4s 31ms/step - loss: 0.0043 - accuracy: 1.0000 - val_loss: 0.3248 - val_accuracy:

Epoch 168/500
133/133 [==============================] - 5s 35ms/step - loss: 0.1028 - accuracy: 0.9850 - val_loss: 0.4224 - val_accuracy: 0.7647
Epoch 169/500
133/133 [==============================] - 4s 33ms/step - loss: 0.0885 - accuracy: 0.9774 - val_loss: 0.4372 - val_accuracy: 0.7059
Epoch 170/500
133/133 [==============================] - 4s 31ms/step - loss: 0.0441 - accuracy: 1.0000 - val_loss: 0.5367 - val_accuracy: 0.7059
Epoch 171/500
133/133 [==============================] - 4s 30ms/step - loss: 0.0390 - accuracy: 0.9925 - val_loss: 0.4795 - val_accuracy: 0.7059
Epoch 172/500
133/133 [==============================] - 4s 30ms/step - loss: 0.0270 - accuracy: 1.0000 - val_loss: 0.3783 - val_accuracy: 0.7647
Epoch 173/500
133/133 [==============================] - 4s 30ms/step - loss: 0.0240 - accuracy: 1.0000 - val_loss: 0.4561 - val_accuracy: 0.7059
Epoch 174/500
133/133 [==============================] - 4s 30ms/step - loss: 0.0184 - accuracy: 1.0000 - val_loss: 0.3917 -

133/133 [==============================] - 4s 30ms/step - loss: 4.0788e-05 - accuracy: 1.0000 - val_loss: 0.3836 - val_accuracy: 0.8235
Epoch 224/500
133/133 [==============================] - 4s 30ms/step - loss: 3.8299e-05 - accuracy: 1.0000 - val_loss: 0.4561 - val_accuracy: 0.7941
Epoch 225/500
133/133 [==============================] - 4s 30ms/step - loss: 2.4565e-05 - accuracy: 1.0000 - val_loss: 0.3907 - val_accuracy: 0.8529
Epoch 226/500
133/133 [==============================] - 4s 30ms/step - loss: 1.3121e-05 - accuracy: 1.0000 - val_loss: 1.2379 - val_accuracy: 0.7941
Epoch 227/500
133/133 [==============================] - 4s 30ms/step - loss: 1.6935e-04 - accuracy: 1.0000 - val_loss: 0.5442 - val_accuracy: 0.8824
Epoch 228/500
133/133 [==============================] - 4s 30ms/step - loss: 6.2713e-05 - accuracy: 1.0000 - val_loss: 0.8842 - val_accuracy: 0.7941
Epoch 229/500
133/133 [==============================] - 5s 35ms/step - loss: 5.4308e-06 - accuracy: 1.0000 - val_

133/133 [==============================] - 4s 30ms/step - loss: 0.0026 - accuracy: 1.0000 - val_loss: 0.6418 - val_accuracy: 0.8235
Epoch 279/500
133/133 [==============================] - 4s 30ms/step - loss: 0.0024 - accuracy: 1.0000 - val_loss: 0.4897 - val_accuracy: 0.7647
Epoch 280/500
133/133 [==============================] - 4s 30ms/step - loss: 0.0023 - accuracy: 1.0000 - val_loss: 0.5060 - val_accuracy: 0.7941
Epoch 281/500
133/133 [==============================] - 4s 30ms/step - loss: 0.0013 - accuracy: 1.0000 - val_loss: 0.4966 - val_accuracy: 0.7647
Epoch 282/500
133/133 [==============================] - 4s 30ms/step - loss: 9.6795e-04 - accuracy: 1.0000 - val_loss: 0.4887 - val_accuracy: 0.7353
Epoch 283/500
133/133 [==============================] - 4s 30ms/step - loss: 0.0015 - accuracy: 1.0000 - val_loss: 0.5058 - val_accuracy: 0.7353
Epoch 284/500
133/133 [==============================] - 4s 30ms/step - loss: 8.3324e-04 - accuracy: 1.0000 - val_loss: 0.5096 - val_a

133/133 [==============================] - 4s 30ms/step - loss: 0.0055 - accuracy: 1.0000 - val_loss: 0.2779 - val_accuracy: 0.8529
Epoch 334/500
133/133 [==============================] - 4s 30ms/step - loss: 0.0030 - accuracy: 1.0000 - val_loss: 0.2866 - val_accuracy: 0.7941
Epoch 335/500
133/133 [==============================] - 4s 31ms/step - loss: 0.0030 - accuracy: 1.0000 - val_loss: 0.2963 - val_accuracy: 0.7941
Epoch 336/500
133/133 [==============================] - 4s 30ms/step - loss: 0.0019 - accuracy: 1.0000 - val_loss: 0.2797 - val_accuracy: 0.8824
Epoch 337/500
133/133 [==============================] - 4s 30ms/step - loss: 0.0014 - accuracy: 1.0000 - val_loss: 0.2837 - val_accuracy: 0.8824
Epoch 338/500
133/133 [==============================] - 4s 30ms/step - loss: 0.0010 - accuracy: 1.0000 - val_loss: 0.2838 - val_accuracy: 0.8824
Epoch 339/500
133/133 [==============================] - 4s 30ms/step - loss: 0.0010 - accuracy: 1.0000 - val_loss: 0.2948 - val_accuracy:

Epoch 388/500
133/133 [==============================] - 4s 31ms/step - loss: 2.4154e-06 - accuracy: 1.0000 - val_loss: 0.4381 - val_accuracy: 0.8824
Epoch 389/500
133/133 [==============================] - 4s 31ms/step - loss: 2.1457e-06 - accuracy: 1.0000 - val_loss: 0.5376 - val_accuracy: 0.8529
Epoch 390/500
133/133 [==============================] - 4s 30ms/step - loss: 1.2620e-06 - accuracy: 1.0000 - val_loss: 0.5268 - val_accuracy: 0.8529
Epoch 391/500
133/133 [==============================] - 4s 31ms/step - loss: 2.9711e-06 - accuracy: 1.0000 - val_loss: 0.4700 - val_accuracy: 0.8235
Epoch 392/500
133/133 [==============================] - 4s 32ms/step - loss: 4.4647e-06 - accuracy: 1.0000 - val_loss: 0.4583 - val_accuracy: 0.8824
Epoch 393/500
133/133 [==============================] - 5s 36ms/step - loss: 2.6941e-06 - accuracy: 1.0000 - val_loss: 0.4510 - val_accuracy: 0.8824
Epoch 394/500
133/133 [==============================] - 4s 31ms/step - loss: 3.9975e-07 - accuracy:

Epoch 443/500
133/133 [==============================] - 4s 31ms/step - loss: 3.8541e-07 - accuracy: 1.0000 - val_loss: 0.7033 - val_accuracy: 0.8235
Epoch 444/500
133/133 [==============================] - 4s 31ms/step - loss: 4.8042e-07 - accuracy: 1.0000 - val_loss: 0.7038 - val_accuracy: 0.8235
Epoch 445/500
133/133 [==============================] - 4s 31ms/step - loss: 7.0802e-06 - accuracy: 1.0000 - val_loss: 0.6915 - val_accuracy: 0.8235
Epoch 446/500
133/133 [==============================] - 4s 32ms/step - loss: 2.5928e-06 - accuracy: 1.0000 - val_loss: 0.6953 - val_accuracy: 0.8235
Epoch 447/500
133/133 [==============================] - 4s 33ms/step - loss: 9.6170e-07 - accuracy: 1.0000 - val_loss: 0.6981 - val_accuracy: 0.8235
Epoch 448/500
133/133 [==============================] - 4s 33ms/step - loss: 3.2536e-07 - accuracy: 1.0000 - val_loss: 0.6992 - val_accuracy: 0.8235
Epoch 449/500
133/133 [==============================] - 4s 31ms/step - loss: 2.4181e-06 - accuracy:

133/133 [==============================] - 5s 37ms/step - loss: 4.0206e-05 - accuracy: 1.0000 - val_loss: 0.1777 - val_accuracy: 0.9412
Epoch 499/500
133/133 [==============================] - 5s 34ms/step - loss: 0.0033 - accuracy: 1.0000 - val_loss: 0.1968 - val_accuracy: 0.9706
Epoch 500/500
133/133 [==============================] - 4s 34ms/step - loss: 3.5495e-05 - accuracy: 1.0000 - val_loss: 0.1969 - val_accuracy: 0.9706


In [41]:
predicted = model.predict(X_test, verbose=1)

predictionsPlot = np.zeros(y_test.shape[0],)

y_testPlot = np.zeros(y_test.shape[0],)

42/42 [==============================] - 0s 5ms/step


In [45]:
from sklearn.metrics import accuracy_score
print(y_test.shape)
print(predicted.shape)
y_testPlot *= 100
predictionsPlot *= 100
acc = accuracy_score(y_testPlot, predictionsPlot)
print("Accuracy: {}".format(acc))

(42, 2)
(42, 2)
Accuracy: 1.0
